In [1]:
#Module Imports
import math
import scipy as sp
from scipy import stats

In [2]:
'''
    DEFINITION OF VARIABLES
    S0 - Stock Price at T=0
    E - Strike Price
    T - Time in Years
    R - Risk Free Rate
'''
S0 = 100
E=100
T=1
R=0.005276

In [3]:
'''BSM VANILLA EUROPEAN OPTION VALUE CALCULATION'''
def bsm_option_value(S0, E, T, R, SIGMA):   
    S0 = float(S0)
    d1 = (math.log(S0/E)+(R+0.05*SIGMA**2)*T)/(SIGMA*math.sqrt(T))
    d2 = d1-(SIGMA*math.sqrt(T))
    
    call_value = S0*stats.norm.cdf(d1,0,1) - E*math.exp(-R*T)*stats.norm.cdf(d2,0,1)
    put_value =  E*math.exp(-R*T)*stats.norm.cdf(-d2,0,1) - (S0*stats.norm.cdf(-d1,0,1))
    return d1, call_value, put_value

In [4]:
def call_implied_vol(call_price):
    CALL_SIGMA = 1  #initial estimate for sigma
    count = 10000  #number of attempts to find value

    #first attempt to set parameters
    d1, call_value, put_value = bsm_option_value(S0, E, T, R, CALL_SIGMA)
    var = abs(call_value - call_price)

    while var > .003 and count > 0:
        VEGA = math.sqrt(T)*S0*stats.norm.pdf(d1,0,1)
        CALL_SIGMA = CALL_SIGMA - ((call_value / VEGA)/100)
        d1, call_value, put_value = bsm_option_value(S0, E, T, R, CALL_SIGMA)
        var = abs((call_value - call_price)/call_value)
        count = count - 1  
    return CALL_SIGMA

In [5]:
def put_implied_vol(put_price):
    PUT_SIGMA = 1 #initial estimate for sigma
    count = 10000  #number of attempts to find value

    #first attempt to set parameters
    d1, call_value, put_value = bsm_option_value(S0, E, T, R, PUT_SIGMA)
    var = abs(put_value - put_price)

    while var > .003 and count > 0:
        VEGA = math.sqrt(T)*S0*stats.norm.pdf(d1,0,1)
        PUT_SIGMA = PUT_SIGMA - ((put_value / VEGA)/100)
        d1, call_value, put_value = bsm_option_value(S0, E, T, R, PUT_SIGMA)
        var = abs((put_value - put_price)/put_value)
        count = count - 1
    return PUT_SIGMA

In [6]:
#input observed call option price to calculate implied volatility
call_price = 28.00
CALL_SIGMA = call_implied_vol(call_price)

print("Implied Volatility %.4f" %CALL_SIGMA)

Implied Volatility 0.7540


In [7]:
#input observed put option price to calculate implied volatility
put_price = 17.00
PUT_SIGMA = put_implied_vol(put_price)

print("Implied Volatility %.4f" %PUT_SIGMA)

Implied Volatility 0.4470


In [8]:
#Validation - Calculates BSM Values using the calculated implied volatility
d1, call_value, put_value = bsm_option_value(S0, E, T, R, CALL_SIGMA)
print("BSM Call Value %.2f" %call_value)
call_diff = call_value - call_price
print("Variance to Root Finding %.3f" %call_diff)
d1, call_value, put_value = bsm_option_value(S0, E, T, R, PUT_SIGMA)
print("BSM Put Value %.2f" %put_value)
put_diff = put_value - put_price
print("Variance to Root Finding %.3f" %put_diff)


BSM Call Value 28.00
Variance to Root Finding 0.002
BSM Put Value 17.03
Variance to Root Finding 0.028


In [9]:
#scipy root finding function
from scipy import optimize

In [10]:
def optimize_function_call(CALL_SIGMA):
    d1, call_value, put_value = bsm_option_value(S0, E, T, R, CALL_SIGMA)
    var = abs(call_value - call_price)
    return var

In [11]:
def optimize_function_put(PUT_SIGMA):
    d1, call_value, put_value = bsm_option_value(S0, E, T, R, PUT_SIGMA)
    var = abs(put_value - put_price)
    return var

In [12]:
opt_call = sp.optimize.fsolve(optimize_function_call, 1)
print("Implied Volatility %.4f" %opt_call[0])

Implied Volatility 0.7540


In [13]:
opt_put = sp.optimize.fsolve(optimize_function_put, 1)
print("Implied Volatility %.4f" %opt_put[0])

Implied Volatility 0.4462
